In [17]:
!pip install pyspark
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [18]:
from pyspark.sql import SparkSession
import findspark

findspark.init()

spark = SparkSession.builder.appName("Pyspark Tutorial")\
                    .config("spark.memory.offHeap.enabled","true")\
                    .config("spark.memory.offHeap.size","10g")\
                    .getOrCreate()


# check spark configuration

spark

In [90]:
!curl https://raw.githubusercontent.com/Sepay/Programming/main/obesity.csv >> obesity.csv
    
df = spark.read.csv('obesity.csv',header=True,sep=",")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  257k  100  257k    0     0   641k      0 --:--:-- --:--:-- --:--:--  642k


In [91]:
df = df.withColumnRenamed('family_history_with_overweight', 'FH')

In [92]:
df.show(5,0)

+------+---+------+------+---+----+----+---+---------+-----+----+---+---+---+----------+---------------------+-------------------+
|Gender|Age|Height|Weight|FH |FAVC|FCVC|NCP|CAEC     |SMOKE|CH2O|SCC|FAF|TUE|CALC      |MTRANS               |NObeyesdad         |
+------+---+------+------+---+----+----+---+---------+-----+----+---+---+---+----------+---------------------+-------------------+
|Female|21 |1.62  |64    |yes|no  |2   |3  |Sometimes|no   |2   |no |0  |1  |no        |Public_Transportation|Normal_Weight      |
|Female|21 |1.52  |56    |yes|no  |3   |3  |Sometimes|yes  |3   |yes|3  |0  |Sometimes |Public_Transportation|Normal_Weight      |
|Male  |23 |1.8   |77    |yes|no  |2   |3  |Sometimes|no   |2   |no |2  |1  |Frequently|Public_Transportation|Normal_Weight      |
|Male  |27 |1.8   |87    |no |no  |3   |3  |Sometimes|no   |2   |no |2  |0  |Frequently|Walking              |Overweight_Level_I |
|Male  |22 |1.78  |89.8  |no |no  |2   |1  |Sometimes|no   |2   |no |0  |0  |Someti

In [93]:
df.count()

21119

In [94]:
df= df.drop('TUE')

In [95]:
df.show()

+------+---+------+------+---+----+----+---+----------+-----+----+---+---+----------+--------------------+-------------------+
|Gender|Age|Height|Weight| FH|FAVC|FCVC|NCP|      CAEC|SMOKE|CH2O|SCC|FAF|      CALC|              MTRANS|         NObeyesdad|
+------+---+------+------+---+----+----+---+----------+-----+----+---+---+----------+--------------------+-------------------+
|Female| 21|  1.62|    64|yes|  no|   2|  3| Sometimes|   no|   2| no|  0|        no|Public_Transporta...|      Normal_Weight|
|Female| 21|  1.52|    56|yes|  no|   3|  3| Sometimes|  yes|   3|yes|  3| Sometimes|Public_Transporta...|      Normal_Weight|
|  Male| 23|   1.8|    77|yes|  no|   2|  3| Sometimes|   no|   2| no|  2|Frequently|Public_Transporta...|      Normal_Weight|
|  Male| 27|   1.8|    87| no|  no|   3|  3| Sometimes|   no|   2| no|  2|Frequently|             Walking| Overweight_Level_I|
|  Male| 22|  1.78|  89.8| no|  no|   2|  1| Sometimes|   no|   2| no|  0| Sometimes|Public_Transporta...|Overw

# Variáveis

Retirou-se a coluna dos telemoveis 

# TRATAMENTO DE DADOS

Alterou-se as colunas Gender, FH, CAEC, SMOLE, SCC, CALC, MTRANS, OBESITY para inteiros para ser mais facil executar os algoritmos


In [96]:
df.groupBy('NObeyesdad').count().show()

+-------------------+-----+
|         NObeyesdad|count|
+-------------------+-----+
|   Obesity_Type_III| 3240|
| Overweight_Level_I| 2900|
|    Obesity_Type_II| 2970|
|Insufficient_Weight| 2720|
|Overweight_Level_II| 2900|
|      Normal_Weight| 2870|
|         NObeyesdad|    9|
|     Obesity_Type_I| 3510|
+-------------------+-----+



In [97]:
from pyspark.sql.functions import * 

df = df.withColumn("CAEC", when(col("CAEC")== "Sometimes", 0).when(col("CAEC") == "Frequently", 1).otherwise(2))
df = df.withColumn("Gender", when(col("Gender")== "Female", 0).otherwise(1))
df = df.withColumn("FH", when(col("FH")== "yes", 0).otherwise(1))
df = df.withColumn("FAVC", when(col("FAVC")== "yes", 0).otherwise(1))
df = df.withColumn("SMOKE", when(col("SMOKE")== "yes", 0).otherwise(1))
df = df.withColumn("SCC", when(col("SCC")== "yes", 0).otherwise(1))
df = df.withColumn("CALC", when(col("CALC")== "no", 0).when(col("CALC") == "Sometimes", 1).when(col("CALC")=="Frequently", 2).otherwise(3))
df = df.withColumn("MTRANS", when(col("MTRANS")== "Public_Transportation", 0).when(col("MTRANS") == "Automobile", 0).when(col("MTRANS")=="Motorbike", 0).otherwise(1))
df = df.withColumn("NObeyesdad", when(col("NObeyesdad")== "Insufficient_Weight", 0).when(col("NObeyesdad") == "Normal_Weight", 1).when(col("NObeyesdad")=="Overweight_Level_I", 2).when(col("NObeyesdad")== "Overweight_Level_II",3).otherwise(4))
df = df.withColumn("FCVC", when(col("FCVC")== null, 2))

In [98]:
df = df.withColumnRenamed("family_history_with_overweight","Family_HIST")
df = df.withColumnRenamed('FAVC','CALORICFOOD')
df=df.withColumnRenamed('Gender','GENDER')
df=df.withColumnRenamed('Age','AGE')
df=df.withColumnRenamed('Weight','WEIGHT')
df=df.withColumnRenamed('Height','HEIGHT')
df=df.withColumnRenamed('FCVC','VEGET')
df=df.withColumnRenamed('NCP','MEALS')
df=df.withColumnRenamed('CAEC','BETWEENMEALS')
df=df.withColumnRenamed('CH2O','H20')
df=df.withColumnRenamed('SCC','CALORIES')
df=df.withColumnRenamed('FAF','PHYSY')
df=df.withColumnRenamed('CALC','ALCH')
df=df.withColumnRenamed('MTRANS','TRANSPORT')
df=df.withColumnRenamed('NObeyesdad','OBESITY')

In [99]:
df.show()

+------+---+------+------+---+-----------+-----+-----+------------+-----+---+--------+-----+----+---------+-------+
|GENDER|AGE|HEIGHT|WEIGHT| FH|CALORICFOOD|VEGET|MEALS|BETWEENMEALS|SMOKE|H20|CALORIES|PHYSY|ALCH|TRANSPORT|OBESITY|
+------+---+------+------+---+-----------+-----+-----+------------+-----+---+--------+-----+----+---------+-------+
|     0| 21|  1.62|    64|  0|          1| null|    3|           0|    1|  2|       1|    0|   0|        0|      1|
|     0| 21|  1.52|    56|  0|          1| null|    3|           0|    0|  3|       0|    3|   1|        0|      1|
|     1| 23|   1.8|    77|  0|          1| null|    3|           0|    1|  2|       1|    2|   2|        0|      1|
|     1| 27|   1.8|    87|  1|          1| null|    3|           0|    1|  2|       1|    2|   2|        1|      2|
|     1| 22|  1.78|  89.8|  1|          1| null|    1|           0|    1|  2|       1|    0|   1|        0|      3|
|     1| 29|  1.62|    53|  1|          0| null|    3|           0|    1

Converter tudo para int

In [100]:
from pyspark.sql.types import StringType,BooleanType,DateType, IntegerType
df = df.withColumn("VEGET",col("VEGET").cast(IntegerType())) \
    .withColumn("MEALS",col("MEALS").cast(IntegerType())) \
    .withColumn("H20",col("H20").cast(IntegerType())) \
    .withColumn("PHYSY",col("PHYSY").cast(IntegerType()))
df.printSchema()

root
 |-- GENDER: integer (nullable = false)
 |-- AGE: string (nullable = true)
 |-- HEIGHT: string (nullable = true)
 |-- WEIGHT: string (nullable = true)
 |-- FH: integer (nullable = false)
 |-- CALORICFOOD: integer (nullable = false)
 |-- VEGET: integer (nullable = true)
 |-- MEALS: integer (nullable = true)
 |-- BETWEENMEALS: integer (nullable = false)
 |-- SMOKE: integer (nullable = false)
 |-- H20: integer (nullable = true)
 |-- CALORIES: integer (nullable = false)
 |-- PHYSY: integer (nullable = true)
 |-- ALCH: integer (nullable = false)
 |-- TRANSPORT: integer (nullable = false)
 |-- OBESITY: integer (nullable = false)



In [101]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

assemble=VectorAssembler(inputCols=[
    'FH','CALORICFOOD', "SMOKE"
], outputCol='features')

In [102]:
assembled_data=assemble.transform(df)

In [103]:
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)

data_scale_output.select('standardized').show(2,truncate=False)

+------------------------------------------+
|standardized                              |
+------------------------------------------+
|[0.0,3.1176544928173704,7.001191599425064]|
|[0.0,3.1176544928173704,0.0]              |
+------------------------------------------+
only showing top 2 rows

